### Generates Provider Features using NPI Database

#### List of changes from ehealey/omop-learn/Manuscript_ready_files/NPI_query
Deleted the commented out code from old

I modified the NPI query to get a valid Json by changing the NPPES NPI api address

In [4]:
## This notebook reads the in_name df from the database, and writes to the out_name database
in_name= 'manuscript_covariates_0_final'
out_name = 'manuscript_covariates_1_final'

In [5]:
import sys
import pickle
import requests
import json
sys.path.append( '..' )
import psycopg2
import pandas as pd
import numpy
#import sparse
import config
import sys
import time
import importlib
#
import datetime
import pandas as pd
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt



In [7]:
import sparse
import Utils.dbutils as dbutils
import Utils.data_utils as data_utils
import Generators.CohortGenerator as CohortGenerator
import Generators.FeatureGenerator as FeatureGenerator
import config
local_imports = (
    dbutils,
    data_utils,
    CohortGenerator,
    FeatureGenerator,
    config
)
for i in local_imports:
    i = importlib.reload(i)

# database connection
username = config.PG_USERNAME
password = config.PG_PASSWORD
database_name = config.DB_NAME

#config_path = 'postgresql://{username}:{password}@{database_name}'.format(
config_path = 'postgresql://{database_name}'.format(database_name = database_name)

connect_args = {"host": '/var/run/postgresql/'} # connect_args to pass to sqlalchemy create_engine function

# schemas 
# schema_name = 'eol_test_ncjones' # all created tables will be created using this schema
cdm_schema_name = config.OMOP_CDM_SCHEMA # the name of the schema housing your OMOP CDM tables
print(f"cdm schema: {cdm_schema_name}")

    #username = username,
    #password = password,
   # database_name = database_name
#)

# schemas 
schema_name = 'eol_test_eheal' # all created tables will be created using this schema

# caching
reset_schema = False # if true, rebuild all data from scratch

# set up database, reset schemas as needed
db = dbutils.Database(config_path, schema_name, connect_args, cdm_schema_name)
# if reset_schema:
#     db.execute(
#         'drop schema if exists {} cascade'.format(schema_name)
#     )
# db.execute(
#     'create schema if not exists {}'.format(schema_name)
# )

cdm schema: cdm_6871_21


In [8]:
# set input
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)

sql_cmd = 'select * from cdm_6871_21.{}'.format(in_name)
covariate_df =pd.read_sql(sql_cmd, conn)

 

## Add provider info

In [10]:
%%time


# visit provider info
sql_cmd = '''
SELECT provider.provider_id, specialty_concept_id, concept_name, npi
FROM (SELECT * FROM cdm_6871_21.provider
        WHERE provider_id IN
            (SELECT visit_provider_id FROM cdm_6871_21.cohort_manuscript_updated_treatments_ncj)) as provider
JOIN (SELECT concept_id, concept_name FROM cdm_6871_21.concept) as concept
ON provider.specialty_concept_id=concept.concept_id
GROUP BY provider.provider_id, provider.specialty_concept_id, concept.concept_name, provider.npi
'''
provider_visit = pd.read_sql(sql_cmd, conn)
provider_visit.head()
print(len(provider_visit))


8606
CPU times: user 23 ms, sys: 3.75 ms, total: 26.7 ms
Wall time: 1.58 s


In [11]:
%%time 
# durg provider infor
sql_cmd = '''
SELECT provider.provider_id, specialty_concept_id, concept_name, npi
FROM (SELECT * FROM cdm_6871_21.provider
        WHERE provider_id IN
            (SELECT provider_id FROM cdm_6871_21.cohort_manuscript_updated_treatments_ncj)) as provider
JOIN (SELECT concept_id, concept_name FROM cdm_6871_21.concept) as concept
ON provider.specialty_concept_id=concept.concept_id
GROUP BY provider.provider_id, provider.specialty_concept_id, concept.concept_name, provider.npi
'''
provider = pd.read_sql(sql_cmd, conn)
provider.head()
print(len(provider))

13149
CPU times: user 38.9 ms, sys: 0 ns, total: 38.9 ms
Wall time: 1.75 s


In [12]:
provider_visit = provider_visit.rename(columns = {'specialty_concept_id': 'visit_specialty_concept_id','provider_id':'visit_provider_id','npi':'visit_npi','concept_name':'visit_concept_name'})
provider = provider.rename(columns = {'npi':'drug_npi'})



In [13]:
#vtemp_visit = v12.join(provider_visit[['visit_provider_id', 'visit_specialty_concept_id', 'visit_concept_name','visit_npi']].set_index('visit_provider_id'), on='visit_provider_id')
vtemp_visit = covariate_df.join(provider_visit[['visit_provider_id', 'visit_specialty_concept_id', 'visit_concept_name','visit_npi']].set_index('visit_provider_id'), on='visit_provider_id')
vtemp_visit['visit_specialty_concept_id'].fillna(0, inplace = True)

vtemp = vtemp_visit.join(provider[['provider_id', 'specialty_concept_id', 'concept_name','drug_npi']].set_index('provider_id'), on='provider_id')
vtemp['specialty_concept_id'].fillna(0, inplace = True)


In [14]:
temp = vtemp[['npi','drug_npi','visit_npi','condition_occurrence_id','person_id','visit_provider_id','provider_id', 'visit_specialty_concept_id', 'specialty_concept_id', 'visit_concept_name','concept_name']]



In [15]:
temp1 = temp.assign(provider_specialty_available = False)
temp1.provider_specialty_available = (~(temp1.concept_name=='No matching concept')&~(temp1.provider_id==-1))

temp1 = temp1.assign(visit_provider_specialty_available = False)
temp1.visit_provider_specialty_available = (~(temp1.visit_concept_name=='No matching concept')&~(temp1.visit_provider_id==-1))
provider_table = temp1.copy()


## Query NPI database and Pickle

In [120]:
%%time
error_index = []
error_condition = []
not_found = []

provider_visit_dict = {}
count = 0
for n in provider_table.visit_npi.unique():
    count+=1
    if count%200==0:
        print(count/provider_table.visit_npi.nunique())
    try:
        r = requests.get(f'https://npiregistry.cms.hhs.gov/api/?version=2.1&number={n}&pretty=on')
        results_text =json.loads(r.text)
        try: 
            provider_visit_dict[n] = results_text["results"][0]["taxonomies"]
        except:
            not_found.append(n)          
    except:
        error_condition.append(n)
        error_index.append(n)
print (f'Number of not found: {len(not_found)}')



0.024280684715308972
0.048561369430617944
0.07284205414592691
0.09712273886123589
0.12140342357654486
0.14568410829185383
0.1699647930071628
0.19424547772247178
0.21852616243778075
0.24280684715308973
0.2670875318683987
0.29136821658370765
0.3156489012990166
0.3399295860143256
0.3642102707296346
0.38849095544494355
0.41277164016025253
0.4370523248755615
0.4613330095908705
0.48561369430617946
0.5098943790214884
0.5341750637367974
0.5584557484521063
0.5827364331674153
0.6070171178827243
0.6312978025980333
0.6555784873133422
0.6798591720286512
0.7041398567439602
0.7284205414592692
0.7527012261745781
0.7769819108898871
0.8012625956051961
0.8255432803205051
0.849823965035814
0.874104649751123
0.898385334466432
0.922666019181741
0.9469467038970499
0.9712273886123589
0.9955080733276679
Number of not found: 124


In [121]:
import pickle
pickle.dump(provider_visit_dict, open('provider_visit_dict.p','wb'))
pickle.dump(not_found, open('provider_visit_npis_not_found.p','wb'))

In [122]:
%%time
# Load Drug NPI information
error_index_drug = []
error_condition_drug = []
not_found_drug = []

provider_drug_dict = {}
count = 0
for n in provider_table.drug_npi.unique():
    count+=1
    if count%600==0:
        print(count/provider_table.drug_npi.nunique())
    try:
        r = requests.get(f'https://npiregistry.cms.hhs.gov/api/?version=2.1&number={n}&pretty=on')
        results_text =json.loads( r.text)
        try: 
            provider_drug_dict[n] = results_text["results"][0]["taxonomies"]
        except:
            not_found_drug.append(n)          
    except:
        error_condition_drug.append(n)
        error_index_drug.append(n)
import pickle
pickle.dump(provider_drug_dict, open('provider_drug_dict.p','wb'))
pickle.dump(not_found_drug, open('provider_drug_npis_not_found.p','wb'))


0.05676442762535478
0.11352885525070956
0.17029328287606432
0.22705771050141912
0.28382213812677387
0.34058656575212864
0.3973509933774834
0.45411542100283825
0.510879848628193
0.5676442762535477
0.6244087038789026
0.6811731315042573
0.7379375591296121
0.7947019867549668
0.8514664143803217
0.9082308420056765
0.9649952696310312


## Load pickle and add information to covariate table

In [126]:
provider_visit_dict =pickle.load(open('provider_visit_dict.p','rb'))
provider_drug_dict =pickle.load(open('provider_drug_dict.p','rb'))

In [ ]:
%%time
# Drug
provider_table = provider_table.assign(visit_npi_desc = '')
provider_table = provider_table.assign(drug_npi_desc = '')
provider_table = provider_table.assign(npi_same = False)

error_index = []
error_condition = []
not_found = []
no_primary_taxonomy_found_drug = []
no_primary_taxonomy_found_visit = []
flag = False
count = 0
multiple = []
for idx,row in provider_table.iterrows():
    n = row.drug_npi
    if str(n) !='nan':
        try:
            taxonomies =provider_drug_dict[n]
            if len(taxonomies)==1:  
                provider_table.loc[idx,'drug_npi_desc'] = taxonomies[0]["desc"]
            else:
                num_tax = 0
                for tax in taxonomies:
                    
                    multiple.append(n)
                    if tax['primary']==True:
                        provider_table.loc[idx,'drug_npi_desc'] = tax["desc"]
                        num_tax +=1
                        flag = True
                if num_tax>1:
                    print(n)
                if flag ==False:
                    no_primary_taxonomy_found_drug.append(n)
                flag = False
        except:
            provider_table.loc[idx,'drug_npi_desc'] = None
    n = row.visit_npi
    if str(n) != 'nan' and n != None:
        try:
            taxonomies =provider_visit_dict[n]
            
            if len(taxonomies)==1:  
                provider_table.loc[idx,'visit_npi_desc'] = taxonomies[0]["desc"]
            else:
                for tax in taxonomies:
                    if tax['primary']==True:
                        provider_table.loc[idx,'visit_npi_desc'] = tax["desc"]
                        flag = True
                        
                if flag ==False:
                    no_primary_taxonomy_found_visit.append(n)
                flag = False
            if row.drug_npi ==row.visit_npi:
                provider_table.loc[idx,'npi_same'] = True    
            count +=1
        except:
            provider_table.loc[idx,'visit_npi_desc'] = None

        if count%5000 ==0:
            print(count/len(provider_table))


0.08316976612661765
0.1663395322532353
0.24950929837985295
0.3326790645064706
0.41584883063308825
0.4990185967597059


In [163]:
covariate_df_new = covariate_df.join(provider_table[['condition_occurrence_id', 'drug_npi_desc']].set_index('condition_occurrence_id'), on='condition_occurrence_id')

### One hot encoding


In [164]:
covariate_df_new = covariate_df_new.assign(provider_specialty_available = False)
covariate_df_new.provider_specialty_available = (~(covariate_df_new['drug_npi_desc']==''))

In [169]:
%%time
covariate_df_new1 = covariate_df_new1.assign(drug_npi_desc_grouped = '')
none_count = 0
i=0
for idx, row in covariate_df_new1.iterrows():
    if row.drug_npi_desc == None:
        none_count+=1
    else:
        desc = row.drug_npi_desc.lower()
        if 'nurse' in desc or 'physician assistant' in desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'advanced_specialist_group'
        elif ('family' in desc and not ('planning' in desc or 'marriage' in desc) ) or  ('general practice' in desc and not 'dentist' in desc):
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'family_medicine_group'
        elif 'internal medicine' == desc or 'internal medicine geriatric medicine' == desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'internal_medicine_group'
        elif 'emergency' in desc or 'acute' in desc or 'hospital' in desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'emergency/acute_group'
        elif 'urology' in desc and not('neurology' in desc):
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'urology_group'
        elif 'gynecology' in desc or 'OBG' in desc or 'midwife' in desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'OBGYN_group'
        elif len(desc)>0 :
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'other_group'
    if i %3000 ==0:
        print(i/len(covariate_df_new1))
        
    i = i+1

0.0
0.04990185967597059
0.09980371935194118
0.14970557902791176
0.19960743870388237
0.24950929837985295
0.2994111580558235
0.34931301773179413
0.39921487740776473
0.44911673708373534
0.4990185967597059
0.5489204564356766
0.598822316111647
0.6487241757876177
0.6986260354635883
0.7485278951395589
0.7984297548155295
0.8483316144915001
0.8982334741674707
0.9481353338434413
0.9980371935194118
CPU times: user 49.9 s, sys: 207 ms, total: 50.1 s
Wall time: 50.1 s


In [172]:
covariate_df_new1.provider_specialty_available =   (covariate_df_new1.drug_npi_desc_grouped!='')

In [181]:
covariate_df_new1[covariate_df_new1['drug_npi_desc_grouped']=='family_medicine_group']

,level_0,index,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,visit_occurrence_id,...,inappropriate_12_to_24_mo,nitrofurantoin_most_recent,fosfomycin_most_recent,trimethoprim-sulfamethoxazole_most_recent,second_line_most_recent,alternatives_most_recent,inappropriate_most_recent,drug_npi_desc,provider_specialty_available,drug_npi_desc_grouped
3,3,3828990,455153289,658,81902,2019-06-01,2019-06-01,None,None,929640,...,0,1,0,0,0,0,0,Family Medicine,True,family_medicine_group
6,6,243383,84456435,866,81902,2014-06-08,2014-06-08,None,None,933140,...,1,0,0,0,0,0,1,Family Medicine,True,family_medicine_group
8,8,3828990,281528462,903,81902,2016-12-05,2016-12-05,None,None,934418,...,1,0,0,0,0,0,1,Family Medicine,True,family_medicine_group
9,9,115391,165986182,906,81902,2015-10-16,2015-10-16,None,None,934656,...,0,0,0,0,0,0,1,Family Medicine,True,family_medicine_group
10,10,115391,96056668,950,81902,2014-07-24,2014-07-24,None,None,935486,...,0,0,0,0,1,0,0,Family Medicine,True,family_medicine_group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60085,60085,3828990,468437334,31047843,195588,2019-05-28,2019-05-28,None,None,159228938,...,0,0,0,0,0,0,0,Family Medicine,True,family_medicine_group
60087,60087,285902,363166483,32644444,81902,2018-03-10,2018-03-10,None,None,160407367,...,1,0,0,0,0,0,1,Family Medicine,True,family_medicine_group
60088,60088,115391,564022635,31075176,81902,2020-10-06,2020-10-06,None,None,159254959,...,0,1,0,0,0,0,0,Family Medicine,True,family_medicine_group
60089,60089,3828990,326697831,31086860,37018854,2017-09-20,2017-09-20,None,None,159267356,...,1,0,0,0,0,0,1,Family Medicine,True,family_medicine_group


In [182]:
specialties = pd.DataFrame(covariate_df_new1.drug_npi_desc_grouped.value_counts()).index
specialties = specialties.drop('')
specialties_list =[]
for s in list(specialties):
    specialties_list.append('specialty_'+str(s))


In [183]:
temp = covariate_df_new1[['drug_npi_desc_grouped']]
onehot = pd.get_dummies(temp,prefix = 'specialty')

In [186]:
covariate_df_new2 = covariate_df_new1.merge(onehot.loc[:,specialties_list],left_index = True,right_index = True)

In [187]:
covariate_df_new2 = covariate_df_new2.drop(columns = ['drug_npi_desc'])

In [188]:
covariate_df_new2 = covariate_df_new2.drop(columns = ['drug_npi_desc_grouped'])

In [189]:
covariate_df_new2 =covariate_df_new2.drop(columns = 'level_0')

In [190]:
covariate_df_new2.to_sql("{}".format(out_name),con=db.engine, if_exists="replace", schema="cdm_6871_21")


In [193]:
db.execute("GRANT SELECT ON TABLE cdm_6871_21.{} TO cdm_6871_21;".format(out_name))



Executed 1 SQLs


# Stop

## Analysis 

In [194]:
# provider_table[provider_table.drug_npi.isna()].visit_npi_desc.value_counts().head(30)

                                                           4717
Clinical Medical Laboratory                                2877
Family Medicine                                            2180
Internal Medicine                                           894
Emergency Medicine                                          745
General Acute Care Hospital                                 646
Clinic/Center, Urgent Care                                  447
Obstetrics & Gynecology                                     292
Nurse Practitioner, Family                                   89
Urology                                                      85
Pathology, Anatomic Pathology & Clinical Pathology           66
Specialist                                                   56
Pediatrics                                                   42
Hospitalist                                                  39
General Practice                                             38
Obstetrics & Gynecology, Gynecology     

In [132]:
# provider_table[provider_table.drug_npi.isna()]

,index,npi,drug_npi,visit_npi,condition_occurrence_id,person_id,visit_provider_id,provider_id,visit_specialty_concept_id,specialty_concept_id,visit_concept_name,concept_name,provider_specialty_available,visit_provider_specialty_available,visit_npi_desc,drug_npi_desc,npi_same
0,0,None,None,1912177908,400169196,29,4239547,-1,0.0,0.0,No matching concept,None,False,False,Clinic/Center Emergency Care,,False
2,2,None,None,1205183894,465004090,101,4242219,-1,0.0,0.0,No matching concept,None,False,False,Emergency Medicine,,False
4,4,None,None,None,148592964,481,4217804,-1,0.0,0.0,No matching concept,None,False,False,,,False
5,5,None,None,None,247776427,481,4217804,-1,0.0,0.0,No matching concept,None,False,False,,,False
20,20,None,None,1902128721,554869223,9964,4240462,-1,0.0,0.0,No matching concept,None,False,False,Clinic/Center Urgent Care,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79973,83445,None,None,1033107057,301486613,32205287,4261920,-1,0.0,0.0,No matching concept,None,False,False,Obstetrics & Gynecology,,False
79981,83453,None,None,1043565021,356369082,33341244,4288319,-1,0.0,0.0,No matching concept,None,False,False,Clinical Medical Laboratory,,False
79982,83454,None,None,1366462459,436711243,33341244,4255358,-1,0.0,0.0,No matching concept,None,False,False,Family Medicine,,False
79983,83455,None,None,1396131603,540800305,33405322,4714388,-1,38003845.0,0.0,Emergency Medicine,None,False,True,Emergency Medicine,,False


In [133]:
# tmp = provider_table.assign(drug_npi_desc_grouped = '')

# i=0
# for idx, row in tmp.iterrows():
#     desc = row.drug_npi_desc.lower()
#     if 'nurse' in desc or 'physician assistant' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'advanced_specialist_group'
#     elif 'family' in desc and not ('planning' in desc or 'marriage' in desc):
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'family_medicine_group'
#     elif 'internal' in desc or 'general practice' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'internal_medicine_group'
#     elif 'emergency' in desc or 'acute' in desc or 'hospital' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'emergency/acute_group'
#     elif 'urology' in desc and not('neurology' in desc):
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'urology_group'
#     elif 'gynecology' in desc or 'OBG' in desc or 'midwife' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'OBGYN_group'
#     elif len(desc)>0 :
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'other_group'
#     if i %3000 ==0:
#         print(i)
        
#     i = i+1

0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000


In [135]:
# tmp = tmp.assign(visit_npi_desc_grouped = '')

# i=0
# for idx, row in tmp.iterrows():
#     desc = row.visit_npi_desc.lower()
#     if 'nurse' in desc or 'physician assistant' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'advanced_specialist_group'
#     elif 'family' in desc and not ('planning' in desc or 'marriage' in desc):
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'family_medicine_group'
#     elif 'internal' in desc or 'general practice' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'internal_medicine_group'
#     elif 'emergency' in desc or 'acute' in desc or 'hospital' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'emergency/acute_group'
#     elif 'urology' in desc and not('neurology' in desc):
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'urology_group'
#     elif 'gynecology' in desc or 'OBG' in desc or 'midwife' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'OBGYN_group'
#     elif len(desc)>0 :
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'other_group'
#     if i %3000 ==0:
#         print(i)
        
#     i = i+1

0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000


In [157]:
# tmp2 = tmp[tmp.drug_npi_desc_grouped != '']

In [160]:
# tmp3 =  tmp2[tmp2.visit_npi_desc_grouped != '']

In [161]:
# tmp3

,index,npi,drug_npi,visit_npi,condition_occurrence_id,person_id,visit_provider_id,provider_id,visit_specialty_concept_id,specialty_concept_id,visit_concept_name,concept_name,provider_specialty_available,visit_provider_specialty_available,visit_npi_desc,drug_npi_desc,npi_same,drug_npi_desc_grouped,visit_npi_desc_grouped
1,1,1609227529,1609227529,1720007123,543518058,35,4259044,4721823,38003845.0,38004456.0,Emergency Medicine,Internal Medicine,True,True,Emergency Medicine,Student in an Organized Health Care Education/...,False,other_group,emergency/acute_group
6,6,1184719445,1184719445,1184719445,206554012,649,4248018,4248018,38003845.0,38003845.0,Emergency Medicine,Emergency Medicine,True,True,Emergency Medicine,Emergency Medicine,True,emergency/acute_group,emergency/acute_group
7,7,1770577900,1770577900,1770577900,455153289,658,4257346,4257346,0.0,0.0,No matching concept,No matching concept,False,False,Family Medicine,Family Medicine,True,family_medicine_group,family_medicine_group
8,8,1427564830,1427564830,1427564830,565954572,658,4674150,4674150,0.0,0.0,No matching concept,No matching concept,False,False,Registered Nurse Emergency,Registered Nurse Emergency,True,advanced_specialist_group,advanced_specialist_group
9,9,1033165360,1033165360,1033165360,329480918,744,4258814,4258814,38004474.0,38004474.0,Urology,Urology,True,True,Urology,Urology,True,urology_group,urology_group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79985,83457,1235181892,1235181892,1235181892,410361010,34041930,4562136,4562136,38003845.0,38003845.0,Emergency Medicine,Emergency Medicine,True,True,Emergency Medicine,Emergency Medicine,True,emergency/acute_group,emergency/acute_group
79987,83459,1376002444,1376002444,1902809940,547287974,34234398,4483589,4710740,0.0,0.0,No matching concept,No matching concept,False,False,Clinical Medical Laboratory,Nurse Practitioner Family,False,advanced_specialist_group,other_group
79988,83460,1912303033,1912303033,1902809940,374486397,34507232,4483589,4575776,0.0,38004461.0,No matching concept,Obstetrics/Gynecology,True,False,Clinical Medical Laboratory,Obstetrics & Gynecology,False,OBGYN_group,other_group
79989,83461,1922528348,1922528348,1336344399,515586779,34507232,4272423,4651097,0.0,0.0,No matching concept,No matching concept,False,False,Family Medicine,Non-Pharmacy Dispensing Site,False,other_group,family_medicine_group


In [162]:
# df = tmp3[['drug_npi_desc_grouped','visit_npi_desc_grouped']]

In [167]:
# df

,drug_npi_desc_grouped,visit_npi_desc_grouped
1,other_group,emergency/acute_group
6,emergency/acute_group,emergency/acute_group
7,family_medicine_group,family_medicine_group
8,advanced_specialist_group,advanced_specialist_group
9,urology_group,urology_group
...,...,...
79985,emergency/acute_group,emergency/acute_group
79987,advanced_specialist_group,other_group
79988,OBGYN_group,other_group
79989,other_group,family_medicine_group


In [163]:
# final = df.groupby(['drug_npi_desc_grouped','visit_npi_desc_grouped']).size()

In [164]:
# temp = pd.DataFrame(final)

In [166]:
# temp

0
drug_npi_desc_grouped     visit_npi_desc_grouped         
OBGYN_group               OBGYN_group                1500
                          advanced_specialist_group    13
                          emergency/acute_group       153
                          family_medicine_group        36
                          internal_medicine_group      31
                          other_group                2137
                          urology_group                 7
advanced_specialist_group OBGYN_group                  86
                          advanced_specialist_group  2874
                          emergency/acute_group       500
                          family_medicine_group       800
                          internal_medicine_group     421
                          other_group                2765
                          urology_group                36
emergency/acute_group     OBGYN_group                   8
                          advanced_specialist_group    49
                          emergency/acute_group      6274
                          family_medicine_group       198
                          internal_medicine_group     428
                          other_group                1083
                          urology_group                22
family_medicine_group     OBGYN_group                  23
                          advanced_specialist_group    76
                          emergency/acute_group       842
                          family_medicine_group      9948
                          internal_medicine_group     231
                          other_group                7974
                          urology_group                30
internal_medicine_group   OBGYN_group                  11
                          advanced_specialist_group    53
                          emergency/acute_group      1349
                          family_medicine_group       194
                          internal_medicine_group    5499
                          other_group                4039
                          urology_group                52
other_group               OBGYN_group                  31
                          advanced_specialist_group    16
                          emergency/acute_group       304
                          family_medicine_group       392
                          internal_medicine_group     221
                          other_group                2406
                          urology_group                19
urology_group             advanced_specialist_group     4
                          emergency/acute_group        98
                          family_medicine_group        19
                          internal_medicine_group      13
                          other_group                 686
                          urology_group               669

In [165]:
# temp.sort_values(by = (0),ascending = False).head(50)

0
drug_npi_desc_grouped     visit_npi_desc_grouped         
family_medicine_group     family_medicine_group      9948
                          other_group                7974
emergency/acute_group     emergency/acute_group      6274
internal_medicine_group   internal_medicine_group    5499
                          other_group                4039
advanced_specialist_group advanced_specialist_group  2874
                          other_group                2765
other_group               other_group                2406
OBGYN_group               other_group                2137
                          OBGYN_group                1500
internal_medicine_group   emergency/acute_group      1349
emergency/acute_group     other_group                1083
family_medicine_group     emergency/acute_group       842
advanced_specialist_group family_medicine_group       800
urology_group             other_group                 686
                          urology_group               669
advanced_specialist_group emergency/acute_group       500
emergency/acute_group     internal_medicine_group     428
advanced_specialist_group internal_medicine_group     421
other_group               family_medicine_group       392
                          emergency/acute_group       304
family_medicine_group     internal_medicine_group     231
other_group               internal_medicine_group     221
emergency/acute_group     family_medicine_group       198
internal_medicine_group   family_medicine_group       194
OBGYN_group               emergency/acute_group       153
urology_group             emergency/acute_group        98
advanced_specialist_group OBGYN_group                  86
family_medicine_group     advanced_specialist_group    76
internal_medicine_group   advanced_specialist_group    53
                          urology_group                52
emergency/acute_group     advanced_specialist_group    49
advanced_specialist_group urology_group                36
OBGYN_group               family_medicine_group        36
other_group               OBGYN_group                  31
OBGYN_group               internal_medicine_group      31
family_medicine_group     urology_group                30
                          OBGYN_group                  23
emergency/acute_group     urology_group                22
other_group               urology_group                19
urology_group             family_medicine_group        19
other_group               advanced_specialist_group    16
OBGYN_group               advanced_specialist_group    13
urology_group             internal_medicine_group      13
internal_medicine_group   OBGYN_group                  11
emergency/acute_group     OBGYN_group                   8
OBGYN_group               urology_group                 7
urology_group             advanced_specialist_group     4